In [5]:
P = 10**9 + 7
r = 2**32
r2 = r * r % P
Pinv = pow(-P, -1, r)
print(Pinv)
print(r2)
def m_reduce(ab):
  m = ab * Pinv % r
  return (ab + m * P) // r

def m_transform(a):
  return m_reduce(a * r2)
a = 10**5
b = 10**4
a_prim = m_transform(a) # mult a by 2^32
b_prim = m_transform(b) # mult b by 2^32
prod_prim = m_reduce(a_prim * b_prim) # divide a' * b' by 2^32
prod = m_reduce(prod_prim)
print(f"{prod}")

2226617417
582344008
1000000000


Montgomery transform:
Idea (easy case)
Suppose that a⋅b
 just happens to be divisible by 232
. Then (a⋅b⋅2−32)%P=(a⋅b)≫32
, which runs super fast!

Idea (general case)
Can we do something similar if a⋅b
 is not divisible by 232
? The answer is yes! The trick is to find some integer m
 such that (a⋅b+m⋅P)
 is divisible by 232
. Then a⋅b⋅2−32%P=(a⋅b+m⋅P)⋅2−32%P=(a⋅b+m⋅P)≫32
.

So how do we find such an integer m
? We want (a⋅b+m⋅P)%232=0
 so m=(−a⋅b⋅P−1)%232
. So if we precalculate (−P−1)%232
 then calculating m
 can be done blazingly fast.

Montgomery transformation
Since the Montgomery reduction divides a⋅b
 by 232
, we would like some some way of multiplying by 232
 modulo P
. The operation x⋅232%P
 is called the Montgomery transform of x
, denoted by m_transform(x)
.

The trick to implement m_transform
 efficently is to make use of the Montgomery reduction. Note that m_transform(x)=m_reduce(x⋅(264%P))
, so if we precalculate 264%P
, then m_transform
 also runs blazingly fast.



Barrett Transform:
Algorithm
Precomputation:

Assume the modulus n∈N
 is such that n≥3
 and n
 is not a power of 2.
(This is for the sake of the proof below, and because modulo-power-of-2 is trivial.)

Choose k∈N
 such that 2k>n
. (The smallest choice is k=⌈log2n⌉
.)

Calculate r=⌊4k/n⌋
. (This is the precomputed factor.)

Reduction:

We are given x∈N
, such that 0≤x<n2
, as the number that needs to be reduced modulo n
.

Calculate t=x−⌊xr4k⌋n
.

If t<n
 then return t
, else return t−n
. This answer is equal to x mod n
.



In [11]:
#my code
import numpy as np
n = 35
k = np.ceil(np.log2(n))
r = np.ceil((4**k)//n)
x = 200
t = x - (np.floor(x*r/4**k))*n
if t<n :
    print(t)
else:
    print(t-n)

25.0


Computing NTT:


In [ ]:
import numpy as np

def number_theoretic_transform(f, p, omega):
    n = len(f)
    if n == 1:
        return f

    f_even = number_theoretic_transform(f[::2], p, (omega * omega) % p)
    f_odd = number_theoretic_transform(f[1::2], p, (omega * omega) % p)

    w = 1
    n_half = n // 2
    result = [0] * n

    for i in range(n_half):
        even = f_even[i]
        odd = (f_odd[i] * w) % p

        result[i] = (even + odd) % p
        result[i + n_half] = (even - odd) % p

        w = (w * omega) % p

    return result


def inverse_number_theoretic_transform(f, p, omega_inv):
    """
    Compute the inverse Number Theoretic Transform (INTT) of a polynomial 'f'
    modulo 'p' using the inverse of the primitive root 'omega_inv'.

    Arguments:
    f -- Input polynomial coefficients in a list format.
    p -- Prime number modulo which the INTT is performed.
    omega_inv -- Inverse of the primitive root of unity modulo 'p'.

    Returns:
    A list of the INTT coefficients.
    """
    n = len(f)
    inv_n = pow(n, p - 2, p)
    f_hat = number_theoretic_transform(f, p, omega_inv)
    f_inv = [(coeff * inv_n) % p for coeff in f_hat]
    return f_inv


# Example usage:
f = [1, 2, 3, 4, 5, 6]  # Coefficients of the polynomial f(x) = 1 + 2x + 3x^2 + 4x^3 + 5x^4 + 6x^5
p = 998244353  # Prime number modulo
omega = 3  # Primitive root of unity modulo 'p'

ntt_coeffs = number_theoretic_transform(f, p, omega)
intt_coeffs = inverse_number_theoretic_transform(ntt_coeffs, p, pow(omega, -1, p))

print("NTT coefficients:", ntt_coeffs)
print("INTT coefficients:", intt_coeffs)


What is a Multiplicative Group?
Group is Set of G,*
It is a group if it is associative, Existence of Identity,Existence of inverse,Closure
If comutative Abelian
Modulo Operation

What are rings?
Groups with two operations
Addition requires Abelian Group Properties
Multiplication requires only Associative Property
They both are required to be distributive

If commutative we call it a commutative ring
with inverses are called division rings

In mathematics, a module is a generalization of the notion of vector space in which the field of scalars is replaced by a ring

What are Fields?
Group +-
Ring +-x
Field +-x÷

has two adittive commutative and multiplicative commutative groups and distributuve property works with both

Finite Field: alois field is a field that contains a finite number of elements. 

Finding Modular inverses 
if we want to know 2 mod 5 's inverse 2 mod 5 * x mod 5 = 1
therefore if take x as 3 we ge so 3 mod 5 is inverse of 2
we find what 1 is congruent to then find we try find x such that x mod 5 * is 1



BOOST Library cross compiled for different rates
